# Sprint22

In [1]:
import numpy as np
import pandas as pd

# ログ表示
class Log:
    # 表示ログレベル
    Loglevel = 0
    def logLevelset(level):
        # 表示ログレベルの設定
        Log.Loglevel = level
    def log_print(level, std, *variable):
        if(Log.Loglevel >= level):
            if(len(variable) == 0):
                print(std)
            else:
                for i in range(len(variable)):
                    index = std.find("}") + 1
                    # 改行なし表示
                    print(std[:index].format(variable[i]), end='')
                    # 表示した分を削除
                    std = std.replace(std[:index], "", 1)
                # 改行表示
                print(std)

In [2]:
Log.logLevelset(1)

データ読込み

In [93]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_sequences, n_nodes)) # ←変更
b = np.array([1])

サンプル実装用のテストコード

FCのフォワード

In [94]:
activation = Tanh()

for batch in range(batch_size):
    for i in range(n_sequences):
        if(i == 0):
            A = np.dot(x[batch, i], w_x) + b
        else:
            A = np.dot(x[batch, i], w_x) + np.dot(h[batch, i-1], w_h) + b
        h[batch,i] = activation.forward(A)

In [95]:
print(np.tanh(A))
print(h)

[0.79494228 0.81839002 0.83939649 0.85584174]
[[[0.76188798 0.76213958 0.76239095 0.76255841]
  [0.792209   0.8141834  0.83404912 0.84977719]
  [0.79494228 0.81839002 0.83939649 0.85584174]]]


In [ ]:
[0.79494228, 0.81839002, 0.83939649, 0.85584174]

In [96]:
activation = Tanh()

for i in range(n_sequences):
    if(i == 0):
        A = np.dot(x[:, i], w_x) + b
    else:
        A = np.dot(x[:, i], w_x) + np.dot(h[:, i-1], w_h) + b
    h[:,i] = activation.forward(A)

In [97]:
np.tanh(A)

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

In [50]:
A.shape

(3, 4)

# 1.スクラッチ

In [100]:
class SimpleRNNClassifier:
    """
    シンプルな三層ニューラルネットワーク分類器

    Parameters
    ----------

    Attributes
    ----------
    """
    def __init__(self, verbose=True, lr=1e-4, num_epoch=2000, batch_size=10, sigma=1e-4, n_nodes1=10, n_nodes2=5, n_output=1):
        self.verbose = verbose
        self.lr = lr
        self.num_epoch = num_epoch # epoch回数
        self.sigma = sigma
        self.n_features = 0
        self.batch_size = 0
        self.n_sequences = 0
        self.n_nodes1 = n_nodes1 # 1層目のノード数
        self.n_nodes2 = n_nodes2 # 2層目のノード数
        self.n_output = n_output # 出力数（3層目のノード数）
        self.cost = []
        
        # レイヤーのインスタンス
        self.FC1 = None
        self.FC2 = None
        self.FC3 = None
        self.activation = None
    
    def fit(self, X, y, X_val=None, y_val=None):
        
        # array変換
        X = np.array(X)
        y = np.array(y)
                
        """
        初期化処理
        """
        self.batch_size = X.shape[0]
        self.n_sequences = X.shape[1]
        self.n_features = X.shape[2]
        
        optimizer = SGD(self.lr)
        self.FC1 = FC(self.n_features, self.n_nodes1, SimpleInitializer(self.sigma), Tanh(), optimizer)
        self.FC2 = FC(self.n_nodes1, self.n_nodes2, SimpleInitializer(self.sigma), Tanh(), optimizer)
        self.FC3 = FC(self.n_nodes2, self.n_output, SimpleInitializer(self.sigma), Tanh(), optimizer)
        self.activation = Tanh()
        
        # エポック数分繰り返す
        for epoch in range(self.num_epoch):
            for i in range(self.n_sequences):
                """
                フォワードプロパゲーション
                """
                A1 = self.FC1.forward(X, i)
                A2 = self.FC2.forward(A1, i)
                A3 = self.FC3.forward(A2, i)
                Z3 = self.activation3.forward(A3)
                Z3 = softmax(Z3)
                
                
                """
                フォワードプロパゲーション
                """
                dA3 = Z3 - y # 交差エントロピー誤差とソフトマックスを合わせている
                dZ2 = self.FC3.backward(dA3)
                dZ1 = self.FC2.backward(dA2)
                dZ0 = self.FC1.backward(dA1) # dZ0は使用しない
                
            # 平均二乗誤差
            loss = MSE.calculation(dA3)
            self.cost.append(loss)
            
    def predict(self, X):
        # array変換
        X = np.array(X)
        
        batch_size = X.shape[0]
        n_sequences = X.shape[1]
        n_features = X.shape[2]
        prdict = np.zeros(n_sequences)
        
        for i in range(n_sequences):
            A1 = self.FC1.forward(X)
            A2 = self.FC2.forward(Z1)
            A3 = self.FC3.forward(Z2)
            prdict[i] = softmax(A3)
            
        return prdict

全結合層のクラス化

In [114]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, activation, optimizer):
        self.activation = activation
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        # 初期化
        # initializerのメソッドを使い、self.W, self.Wh, self.Bを初期化する
        self.W, self.Wh, self.h = initializer.W(n_nodes1, n_nodes2)
        self.B = initializer.B(n_nodes2) 
        self.forward_x = 0
        self.dZ = 0
        self.dA = 0
        
        # AddGrad用
        self.HW = 0
        self.HB = 0
        
    def forward(self, X, n_sequences):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        n_sequences : 現在のサンプル数
        
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """            
        self.forward_x = X.copy()
        
        if(n_sequences == 0):
            A = np.dot(X[:, n_sequences], self.W) + self.B
        else:
            A = np.dot(X[:, n_sequences], self.W) + np.dot(self.h[:, n_sequences-1], self.Wh) + self.B
        
        # Afin
        A = self.activation(A)
        self.h[:, n_sequences] = A
        
        return A
    
    def backward(self, dA, n_sequences):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        n_sequences : 現在のサンプル数
        
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        
        self.dA = dA
        
        self.dZ = np.dot(dA, self.W.T)
        
        # 更新
        self = self.optimizer.update(self, n_sequences)
        
        self.dZ = self.activation.backward(self.dZ)
        
        return self.dZ

初期化方法のクラス化

In [96]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    function : std
      初期関数のモード
    """
    def __init__(self, sigma):
        self.sigma = sigma
        
    def W(self, n_nodes1, n_nodes2, n_samples):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
        n_samples : int
           入力データのサンプル数        

        Returns
        ----------
        W :
        """
        
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        Wh = self.sigma * np.random.randn(n_nodes2, n_nodes1)
        h = np.zeros((n_samples, n_nodes2), dtype = float)
        
        return W, Wh, h
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        
        B = self.sigma * np.random.randn(1, n_nodes2)
        
        return B

In [136]:
class XavierInitializer:
    """
    ザビエル分布による初期化
    Parameters
    ----------
    sigma : floatz
      ガウス分布の標準偏差
    function : std
      初期関数のモード
    """
    def __init__(self):
        self.sigma = 0
        
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        self.sigma = 1 / np.sqrt(n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        Wh = self.sigma * np.random.randn(n_nodes2, n_nodes1)
        
        return W, Wh
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B = self.sigma * np.random.randn(1, n_nodes2)
        
        return B

In [137]:
class HeInitializer:
    """
    He分布による初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    function : std
      初期関数のモード
    """
    def __init__(self):
        self.sigma = 0
        
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        self.sigma = np.sqrt(2 / n_nodes1)
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        Wh = self.sigma * np.random.randn(n_nodes2, n_nodes1)
        
        return W, Wh
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B = self.sigma * np.random.randn(1, n_nodes2)
        
        return B

最適化手法のクラス化

In [128]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer, n_sequences):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        n_sequences : 現在のサンプル数 

        Returns
        ----------
        layer : 更新後の層のインスタンス
        """
        layer.W = layer.W - np.mean(self.lr*np.dot(layer.forward_x.T, layer.dA), axis=0) / layer.dA.shape[0]
        layer.B = layer.B - self.lr*np.mean(layer.dA, axis=0)[np.newaxis,:]
        
        layer.Wh = layer.Wh - np.mean(self.lr*np.dot(layer.h[:,n_sequences-1].T, layer.dA), axis=0) / layer.dA.shape[0]
        
        return layer

In [138]:
class AddGrand:
    """
    最適劣勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
        
    def update(self, layer, n_sequences):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        n_sequences : 現在のサンプル数 

        Returns
        ----------
        layer : 更新後の層のインスタンス
        """
        layer.HW = layer.HW + layer.W**2
        layer.HB = layer.HB + np.mean(layer.B**2, axis=0)[np.newaxis,:]
        
        layer.W = layer.W - self.lr*(1/np.sqrt(layer.HW))*np.dot(layer.forward_x.T, layer.dA) / layer.dA.shape[0]
        layer.B = layer.B - self.lr*(1/np.sqrt(layer.HB))*np.mean(layer.dA, axis=0)[np.newaxis,:]
        
        return layer

活性化関数

In [11]:
class Tanh:
    def __init__(self):
        self.A = 0
        
    def forward(self, A):
        self.A = A
        return np.tanh(A)
    
    def backward(self, dZ):
        dA = dZ *(1 - np.tanh(self.A)**2)
        Log.log_print(1, "dA.shape{}", dA.shape)
        Log.log_print(2, "dA{}", dA)
        return dA

In [139]:
class ReLU:
    def __init__(self):
        self.A = 0
        
    def forward(self, A):
        self.A = A
        return np.maximum(A, 0)
    
    def backward(self, dZ):
        A = self.A.copy()
        A[A>0]=1
        A[A<=0]=0
        dA = dZ *A
        Log.log_print(1, "dA.shape{}", dA.shape)
        Log.log_print(2, "dA{}", dA)
        return dA

恒等関数

In [135]:
class Identity:
    def __init__(self):
        pass
    def forward(self, A):
        return A
    def backward(self, Yhat, Y):
        return Yhat - Y

Softmax

In [98]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T

    x = x - np.max(x)
    return np.exp(x) / np.sum(np.exp(x))

ロス関数

In [134]:
class MSE:
    @classmethod
    def calculation(self, loss):
        return np.mean((loss)**2) / 2

In [99]:
def CrossEntropy(loss, y):
    return - np.mean( np.sum(y.T*np.log(loss), axis=1) )

DropOut

In [2]:
class DropOut:
    def __init__(self):
        self.mask = None
        
    def forward(self, X):
        self.mask = np.random.rand(*X.shape) > 0.5
        return X*self.mask
    
    def backward(self, X):
        return X*self.mask

# 説明

## RNNが通常の全結合層ニューラルネットワークと異なるのはどういった点か
異なるのは時系列データを扱うために、過去の自分自身（同レイヤー）の出力を入力に加算する点である。

## なぜRNNは自然言語処理に使われるのか
単語は文脈によって意味が変わるので、前後の関係を考慮できるRNNが使用される。

## SimpleRNNは勾配消失が起きやすいと言われているがそれはなぜか
RNNの逆伝搬では時間方向の勾配に着目すると逆伝搬によって伝わる勾配は「tanh」を通過することになる。「tanh」ノードは通過するたびに値はどんどん小さくなるため、時系列データが長いものは勾配消失が起きやすい。